# São Paulo Real Estate - Sale/Rent - April 2019

**D2APR: Aprendizado de Máquina e Reconhecimento de Padrões** (IFSP, Campinas)
Especialização em Ciência de Dados - IFSP Campinas

Alunos
- Daniel Vargas Shimamoto
- Diego Machado de Assis

## 1. Informações Gerais

### 1.1. Contexto do problema

### 1.2. Conhecendo  o dataset

O Dataset contem cerca de 13000 dados referentes a venda e aluguel de apartamentos na cidade de São Paulo (Brasil). Os dados foram coletados de diversas fontes, principalmente de sites de classificados de imóveis. Todos os dados foram coletados no mês de abril de 2019.


### 1.3. Principais atributos e seus tipos

Os dados dos imóveis possuem 16 atributos

* Price (int): Preço total anunciado em reais
* Condo (int): Condomínio em reais (Valores desconhecidos são marcados como zero) - Float
* Size (int): Tamanho da propriedade em m² (Somente áreas privadas) - 
* Rooms (int): Número de Quartos - 
* Toilets (int): Númerto total de banheiros - 
* Suites (int): Número de suites (Quartos com banheiros privativos) - 
* Parking (int): Número de vagas de estacionamento
* Elevator (binario): Se existe elevador (1 - sim, 0 - não) - 
* Furnished (binario): Se o imóvel é mobiliado (1 - sim, 0 - não) - 
* Swimming Pool (binario): Se existe piscina na propriedade (1 - sim, 0 - não) - 
* New (binario): Se o apartamento é novo (1 - sim, 0 - não) - 
* District (string) : Bairro e cidade que o imóvel está localizado
* Negotiation Type (string): Alugel ou venda
* Property Type (string): Tipo de propriedade
* Latitude (float): Latitude do imóvel
* Longitude (float): Longitude do imóvel 

In [3]:
import pandas as pd

df = pd.read_csv('../Data/sao-paulo-properties-april-2019.csv')
df.shape

(13640, 16)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13640 entries, 0 to 13639
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Price             13640 non-null  int64  
 1   Condo             13640 non-null  int64  
 2   Size              13640 non-null  int64  
 3   Rooms             13640 non-null  int64  
 4   Toilets           13640 non-null  int64  
 5   Suites            13640 non-null  int64  
 6   Parking           13640 non-null  int64  
 7   Elevator          13640 non-null  int64  
 8   Furnished         13640 non-null  int64  
 9   Swimming Pool     13640 non-null  int64  
 10  New               13640 non-null  int64  
 11  District          13640 non-null  object 
 12  Negotiation Type  13640 non-null  object 
 13  Property Type     13640 non-null  object 
 14  Latitude          13640 non-null  float64
 15  Longitude         13640 non-null  float64
dtypes: float64(2), int64(11), object(3)
memo

## 2. Análise exploratória

### 2.1. Distribuição das variáveis

### 2.2. Correlação

### 2.3. Análise Gráfica

### 2.4. Conclusão

## 3. Limpeza e preparação das Bases

* Remoção de duplicidade e/ou outliers;
* Preenchimento de dados faltantes;
* Feature scaling
* Discussão sucinta sobre a razão de cada etapa de limpeza e pre-processamento considerada;

## 4. Treinamento e validação de modelos de regressão

### 4.1. Modelo 1: xxxx

### 4.2. Modelo 2: xxxx

### 4.3. Modelo 3: xxxx

### 4.4. Discussão dos resultatos
* Métricas utilizadas
* Há overfitting ou underfitting?

### 4.5. Próximos passos
* Estratégias, ideias, sugestões para melhorar o modelo

## Atividades opcionais

### a) Usar pipelines (scikit-Learn)

### b) Fine-tuning

### c) Ensemble methods

### d) Abordagem de negócio
* Motivação e descrição do problema 
* Baseline
* Comparação dos resultados com o baseline
* Conversão dos resultados em medidas de performace para o negócio